In [7]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

"""
Hyperparameters include
1. Model hyperparameters eg number and width of hidden layers
2. Algorithm hyperparameters which influence the speed and quality of the learning algorithm eg learning rate
"""

(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data() # load MNIST dataset

# scale the pixel values to range of 0 to 1 by dividing by 255
# at the same time convert the integers to floating-point numbers
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

def model_builder(hp):
  model = keras.Sequential() # sequential model wher each layer has 1 input tensor and 1 output tensor
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # hypermodel to define the search space for hyperparameters
  # tune the number of units in the first Dense layer
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)  # between 32-512
  model.add(keras.layers.Dense(units=hp_units, activation='relu')) # fully connected layer
  model.add(keras.layers.Dense(10)) 

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) # learning rate bw 0.01, 0.001, or 0.0001

 # Adam optimiser: stochastic gradient descent that maintains per-parameter learning rate 
 # and these are adapted based av of recent magnitudes of gradients for weights
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                # loss function bw ground truth and vector of logits
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                metrics=['accuracy'])

  return model

# instantiate tuner to perform hypertuning
# Hyperband trains a large number of models for a few epochs and carries forward only the top performing 
# half of models to the next round. It determines the number of models to train in a bracket by 
# computing 1 + log(max_epochs) and round up to nearest integer
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5) # stop early training after reaching certain validation loss

In [12]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0] # Get the optimal hyperparameters

INFO:tensorflow:Oracle triggered exit


In [13]:
model = tuner.hypermodel.build(best_hps) # build model with the optimal hyperparameters 
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1 # find optimal number of epochs to train the model with optimal hyperparameters

Epoch 1/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.5013 - accuracy: 0.8221 - val_loss: 0.4515 - val_accuracy: 0.8369
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3728 - accuracy: 0.8650 - val_loss: 0.3516 - val_accuracy: 0.8714
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3330 - accuracy: 0.8769 - val_loss: 0.3582 - val_accuracy: 0.8725
Epoch 4/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3092 - accuracy: 0.8862 - val_loss: 0.3351 - val_accuracy: 0.8767
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2906 - accuracy: 0.8917 - val_loss: 0.3419 - val_accuracy: 0.8780
Epoch 6/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2744 - accuracy: 0.8978 - val_loss: 0.3388 - val_accuracy: 0.8779
Epoch 7/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2591 - accuracy: 0.9037 - val_loss: 0.3170 - val_accuracy:

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps) 
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2) # Retrain the model with optimal number of epochs

# evaluate model
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

Epoch 1/21
1500/1500 [==============================] - 9s 5ms/step - loss: 0.5020 - accuracy: 0.8216 - val_loss: 0.4137 - val_accuracy: 0.8491
Epoch 2/21
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3730 - accuracy: 0.8642 - val_loss: 0.3492 - val_accuracy: 0.8748
Epoch 3/21
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3340 - accuracy: 0.8767 - val_loss: 0.3605 - val_accuracy: 0.8693
Epoch 4/21
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3101 - accuracy: 0.8855 - val_loss: 0.3915 - val_accuracy: 0.8596
Epoch 5/21
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2906 - accuracy: 0.8925 - val_loss: 0.3394 - val_accuracy: 0.8793
Epoch 6/21
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2758 - accuracy: 0.8979 - val_loss: 0.3348 - val_accuracy: 0.8781
Epoch 7/21
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2617 - accuracy: 0.9018 - val_loss: 0.3172 - val_accuracy:

In [ ]:
# apply softmax to the trained model to return probability
probability_model = tf.keras.Sequential([
  hypermodel,
  tf.keras.layers.Softmax()
])
probability_model(img_test[:5])